# Counterfactuals Training Data Extraction Experiment

In [8]:
import pandas as pd
from sklearnex import patch_sklearn
patch_sklearn()
import sklearn.ensemble as es
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import random
import logging
import warnings
import dice_ml

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [9]:
threads = 15

logging.basicConfig()

logger = logging.getLogger('xai-privacy')

In [10]:
from experiment_setup import run_all_experiments
from experiment_setup import get_heart_disease_dataset
from experiment_setup import get_census_dataset

In [11]:
DATASET_HALF = False

data_heart_dict, data_heart_num_dict, data_heart_cat_dict = get_heart_disease_dataset(halve_dataset=DATASET_HALF)
data_census_dict, data_census_num_dict, data_census_cat_dict = get_census_dataset(halve_dataset=DATASET_HALF)

data_heart = data_heart_dict['dataset']
data_heart_num = data_heart_num_dict['dataset']
data_heart_cat = data_heart_cat_dict['dataset']
data_census = data_census_dict['dataset']
data_census_num = data_census_num_dict['dataset']
data_census_cat = data_census_cat_dict['dataset']
outcome_name_heart = data_heart_dict['outcome']
numeric_features_heart = data_heart_dict['num']

Feature Age: removed 0 rows for missing values.
Feature RestingBP: removed 59 rows for missing values.
Feature Cholesterol: removed 27 rows for missing values.
Feature FastingBS: add unknown category 2.0
Feature RestingECG: add unknown category 3.0
Feature MaxHR: removed 0 rows for missing values.
Feature Oldpeak: removed 7 rows for missing values.
Feature ST_Slope: add unknown category 4.0
Feature CA: add unknown category 4.0
Feature Thal: add unknown category 8.0
Dropped 271 of 1097
Dropped 273 of 1097
Dropped 277 of 1097
Dropped: 2399 of 32561
census: Dropped 3848 of 30162
num: Dropped 19859 of 30162
cat: Dropped 12136 of 30162


This notebook will test whether training data extraction is possible with counterfactuals (CF) that are drawn from the training data. Training data extraction means an attacker can find out the feature values of samples from the training data without prior knowledge of them. The attacker only has access to the model's prediction function and the explanation.

This attack should be trivial because any counterfactual that is shown as an explanation was picked directly from the training data.

The idea for counterfacutal training data extraction is as follows: The attacker makes repeated queries to the model with random input values. In order to do this, the attacker knows the maximum and minimum value of each feature in the training data (or the categorical values of each feature). The returned counterfactuals are the extracted training data.

First, we implement the `train_explainer` and `training_data_extraction_model_access` functions:

In [12]:
# Attack code must be imported so that multiprocessing pool works. Check out ice_attack.py for the implementation of the attack.
from cf_attack import CounterfactualTDE

# Executing Training Data Extraction

We now generate five counterfactuals for the first sample from the training data to demonstrate counterfactual explanations in general.

In [13]:
features = data_heart.drop(outcome_name_heart, axis=1)
labels = data_heart[outcome_name_heart]

# Train a random forest on training data.
model = es.RandomForestClassifier(random_state=0)
model = model.fit(features, labels)

# Train explainer
d = dice_ml.Data(dataframe=data_heart, continuous_features=numeric_features_heart, outcome_name=outcome_name_heart)

m = dice_ml.Model(model=model, backend="sklearn", model_type='classifier')
# Generating counterfactuals from training data (kd-tree)
exp = dice_ml.Dice(d, m, method="kdtree")

In [14]:
e1 = exp.generate_counterfactuals(features[0:1], total_CFs=5, desired_class="opposite")
e1.visualize_as_dataframe(display_sparse_df=False)

100%|██████████| 1/1 [00:00<00:00,  1.04it/s]

Query instance (original outcome : 0)


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,CA,Thal,HeartDisease
0,47.0,1.0,1.0,110.0,249.0,0.0,0.0,150.0,0.0,0.0,4.0,4.0,8.0,0.0



Diverse Counterfactual set without sparsity correction (new outcome:  1


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,CA,Thal
388,35.0,1.0,2.0,110.0,257.0,0.0,0.0,140.0,0.0,0.0,4.0,4.0,8.0
434,47.0,1.0,3.0,108.0,243.0,0.0,0.0,152.0,0.0,0.0,1.0,0.0,3.0
705,54.0,1.0,3.0,120.0,237.0,0.0,0.0,150.0,1.0,1.5,4.0,4.0,7.0
320,46.0,1.0,4.0,120.0,249.0,0.0,2.0,144.0,0.0,0.8,1.0,0.0,7.0
1111,33.0,0.0,4.0,100.0,246.0,0.0,0.0,150.0,1.0,1.0,2.0,4.0,8.0


We can see that the counterfactuals are similar to the query sample and that they have a flipped prediction. These are the two general properties of counterfactual explanations.

We will now do a small proof of concept of the experiment with logging enabled to demonstrate how it works.

In [15]:
logger.setLevel(logging.DEBUG)
logging.root.setLevel(logging.ERROR)

EXP = CounterfactualTDE(data_heart, numeric_features_heart, outcome_name_heart, random_state=0)
EXP.training_data_extraction_experiment(num_queries=12, model=es.RandomForestClassifier(random_state=0), model_access=False)

logger.setLevel(logging.INFO)

DEBUG:xai-privacy:Numeric Features: ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']
DEBUG:xai-privacy:Categorical Features: ['CA', 'ChestPainType', 'ExerciseAngina', 'FastingBS', 'RestingECG', 'ST_Slope', 'Sex', 'Thal']
100%|██████████| 12/12 [00:09<00:00,  1.29it/s]
DEBUG:xai-privacy:Sample 0: Counterfactuals 
 [[38.0 '1.0' '4.0' 92.0 117.0 '0.0' '0.0' 134.0 '1.0' 2.5 '2.0' '4.0'
  '8.0']
 [57.0 '1.0' '4.0' 130.0 131.0 '0.0' '0.0' 115.0 '1.0' 1.2 '2.0' '1.0'
  '7.0']
 [51.0 '0.0' '4.0' 120.0 0.0 '2.0' '0.0' 127.0 '1.0' 1.5 '1.0' '4.0'
  '8.0']
 [36.0 '1.0' '4.0' 110.0 0.0 '2.0' '0.0' 125.0 '1.0' 1.0 '2.0' '4.0'
  '6.0']]
DEBUG:xai-privacy:Sample 1: Counterfactuals 
 [[48.0 '1.0' '4.0' 160.0 268.0 '0.0' '0.0' 103.0 '1.0' 1.0 '2.0' '4.0'
  '8.0']
 [57.0 '1.0' '4.0' 150.0 276.0 '0.0' '2.0' 112.0 '1.0' 0.6 '2.0' '1.0'
  '6.0']
 [57.0 '1.0' '4.0' 165.0 289.0 '1.0' '2.0' 124.0 '0.0' 1.0 '2.0' '3.0'
  '7.0']
 [56.0 '0.0' '4.0' 200.0 288.0 '1.0' '2.0' 133.0 '1.0' 4.0 '3.0' '2.0'
  '7.

Total time: 9.74s (training model: 0.05s, training explainer: 0.05s, experiment: 9.64s)
Number of extracted samples: 41
Number of accurate extracted samples: 41
Precision: 1.0, recall: 3.4166666666666665


The proof of concept should show that each extracted sample is an actual training sample (precision of 100%). Recall is above 100% because this method can extract multiple samples per query (multiple counterfactuals are returned). Recall will reach a reasonable value if the experiment is executed for the full training data. In this case, the attack cannot return more samples than the number of queries because the attack is limited by the number of training samples.

Now we begin executing the actual experiment. We begin by defining the table that will hold the results for all our different experiment variations. Then we execute all variations of the experiment for this dataset. We vary the model between a decision tree, a random forest and a neural network. Each model uses the default configuration of scikit-learn.

In [16]:
dataset_dicts = [data_heart_dict, data_heart_num_dict, data_heart_cat_dict, data_census_dict, data_census_num_dict, data_census_cat_dict]

dt_dict = {'name': 'decision tree', 'model': DecisionTreeClassifier}
rf_dict = {'name': 'random forest', 'model': es.RandomForestClassifier}
nn_dict = {'name': 'neural network', 'model': MLPClassifier}

model_dicts = [dt_dict, rf_dict, nn_dict]

# We set the number of extractions to the length of the dataset
num_queries_dict = { 'heart': len(data_heart), 'heart numeric': len(data_heart_num), 'heart categorical': len(data_heart_cat), 'census': len(data_census), 'census numeric': len(data_census_num), 'census categorical': len(data_census_cat)}

In [17]:
# remove pandas warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
# This will run the experiment for each dataset and model combination

results = run_all_experiments(CounterfactualTDE, dataset_dicts, model_dicts, random_state=0, num_queries=num_queries_dict, model_access=False, threads=threads, is_mem_inf=False, convert_cat_to_str=True, repeat=10)

dataset: heart, model: decision tree (repetition 0)
Total time: 72.08s (training model: 0.02s, training explainer: 0.04s, experiment: 72.03s)
Number of extracted samples: 529
Number of accurate extracted samples: 529
Precision: 1.0, recall: 0.6404358353510896
dataset: heart, model: decision tree (repetition 1)
Total time: 70.20s (training model: 0.01s, training explainer: 0.03s, experiment: 70.16s)
Number of extracted samples: 536
Number of accurate extracted samples: 536
Precision: 1.0, recall: 0.648910411622276
dataset: heart, model: decision tree (repetition 2)
Total time: 65.65s (training model: 0.02s, training explainer: 0.03s, experiment: 65.60s)
Number of extracted samples: 506
Number of accurate extracted samples: 506
Precision: 1.0, recall: 0.612590799031477
dataset: heart, model: decision tree (repetition 3)
Total time: 64.55s (training model: 0.02s, training explainer: 0.02s, experiment: 64.51s)
Number of extracted samples: 558
Number of accurate extracted samples: 558
Preci

c:\users\ml_ks\venvs\xai_new\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Total time: 69.01s (training model: 1.95s, training explainer: 0.01s, experiment: 67.05s)
Number of extracted samples: 466
Number of accurate extracted samples: 466
Precision: 1.0, recall: 0.5641646489104116
dataset: heart, model: neural network (repetition 1)


c:\users\ml_ks\venvs\xai_new\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Total time: 67.93s (training model: 1.90s, training explainer: 0.01s, experiment: 66.02s)
Number of extracted samples: 419
Number of accurate extracted samples: 419
Precision: 1.0, recall: 0.5072639225181598
dataset: heart, model: neural network (repetition 2)


...

c:\users\ml_ks\venvs\xai_new\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Total time: 180.24s (training model: 12.37s, training explainer: 0.00s, experiment: 167.87s)
Number of extracted samples: 1161
Number of accurate extracted samples: 1161
Precision: 1.0, recall: 0.11268562554595749
dataset: census categorical, model: decision tree (repetition 0)
Total time: 1043.56s (training model: 0.53s, training explainer: 0.19s, experiment: 1042.84s)
Number of extracted samples: 14220
Number of accurate extracted samples: 14220
Precision: 1.0, recall: 0.788860534783091
dataset: census categorical, model: decision tree (repetition 1)
Total time: 1047.61s (training model: 0.53s, training explainer: 0.17s, experiment: 1046.91s)
Number of extracted samples: 14153
Number of accurate extracted samples: 14153
Precision: 1.0, recall: 0.7851436813491623
dataset: census categorical, model: decision tree (repetition 2)
Total time: 1039.98s (training model: 0.56s, training explainer: 0.20s, experiment: 1039.22s)
Number of extracted samples: 14057
Number of accurate extracted sa

c:\users\ml_ks\venvs\xai_new\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Total time: 1896.08s (training model: 11.03s, training explainer: 0.17s, experiment: 1884.88s)
Number of extracted samples: 13927
Number of accurate extracted samples: 13927
Precision: 1.0, recall: 0.7726062354377011
dataset: census categorical, model: random forest (repetition 1)


c:\users\ml_ks\venvs\xai_new\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


# Results

Precision is the percentage of extracted samples that is actually from the training data. 

Recall is the ratio of the number extracted training samples to all training samples.

In [ ]:
results

In [ ]:
file_name = 'results/2-1-cf-training-data-extraction-results'
if DATASET_HALF:
    file_name += '_dataset_size_halved'
results.to_csv(file_name + '.csv', index=False, na_rep='NaN', float_format='%.3f')

# Discussion

In our experiments, training data extraction with counterfactuals drawn from the training data has a recall between 45% and 67% for numeric data and 30% to 64% for categorical data. Since the attack cannot produce any false positive samples, precision is always 100%.